# AG Grid Examples


In [ ]:
import pandas as pd

import nbappinator as nb
from nbappinator import (
    FORMAT_DECIMAL,
    FORMAT_MAG_SI,
    FORMAT_PERCENT,
    apply_format,
    create_grid,
    get_column_defs,
)

In [ ]:
# Sample data
df = pd.DataFrame(
    {
        "name": ["Apple", "Microsoft", "Google", "Amazon", "Meta"],
        "price": [178.50, 378.91, 141.80, 178.25, 505.95],
        "change_pct": [0.0234, -0.0156, 0.0089, 0.0312, -0.0045],
        "volume": [52_340_000, 18_920_000, 21_450_000, 45_670_000, 12_340_000],
        "market_cap": [2.8e12, 2.9e12, 1.8e12, 1.9e12, 1.3e12],
    }
)

## Standalone Grid

Use `create_grid()` to create a grid widget directly. This is useful when you just want a grid without the full App infrastructure.

In [ ]:
create_grid(df, height=250)

In [ ]:
# Grid with column formatting using get_column_defs and apply_format
# First, generate column definitions from the DataFrame
columns = get_column_defs(df)

# Then customize specific columns
columns[0]["headerName"] = "Company"
columns[0]["pinned"] = "left"

# Apply built-in formatters
apply_format(columns[1], FORMAT_DECIMAL, precision=2)  # price
columns[1]["headerName"] = "Price"

apply_format(columns[2], FORMAT_PERCENT, precision=2)  # change_pct
columns[2]["headerName"] = "Change %"

apply_format(columns[3], FORMAT_MAG_SI, precision=1)  # volume
columns[3]["headerName"] = "Volume"

apply_format(columns[4], FORMAT_MAG_SI, precision=2)  # market_cap
columns[4]["headerName"] = "Market Cap"

grid_formatted = create_grid(df, column_defs=columns, height=250)
grid_formatted

In [ ]:
# Grid with click handler
def on_click(event):
    print(f"Clicked: {event.get('col_clicked')} = {event.get('value_clicked')}")
    print(f"Row data: {event.get('data')}")


grid_clickable = create_grid(
    df,
    column_defs=columns,
    action=on_click,
    height=250,
)
grid_clickable

## Custom JavaScript Formatters

You can add custom formatters by setting `valueFormatter` to a JavaScript arrow function string:

In [ ]:
# Custom JavaScript formatter example
# Use valueFormatter with a JS arrow function string
custom_cols = get_column_defs(df)

# Currency
custom_cols[1]["valueFormatter"] = "params => params.value ? '$' + params.value.toFixed(2) : ''"
custom_cols[1]["headerName"] = "Price (USD)"

# bps
custom_cols[2]["valueFormatter"] = "params => params.value ? (params.value * 10000).toFixed(0) + ' bps' : ''"
custom_cols[2]["headerName"] = "Change (bps)"

# Vol
custom_cols[3]["valueFormatter"] = "params => params.value ? params.value.toLocaleString() + ' shares' : ''"
custom_cols[3]["headerName"] = "Volume"

# Trillions
custom_cols[4]["valueFormatter"] = "params => params.value ? '$' + (params.value / 1e12).toFixed(2) + 'T' : ''"
custom_cols[4]["headerName"] = "Market Cap"

grid_custom_format = create_grid(df, column_defs=custom_cols, height=250)
grid_custom_format

In [ ]:
# Access current selection
print("Current selection:", grid_clickable.current_selection)

In [ ]:
# Pagination

grid_custom = create_grid(
    df,
    column_defs=columns,
    height=250,
    grid_options={
        "pagination": True,
        "paginationPageSize": 3,
        "suppressRowClickSelection": True,
        "enableCellTextSelection": True,
    },
)
grid_custom

In [ ]:
# Use a specific AG Grid version
grid_versioned = create_grid(
    df,
    column_defs=columns,
    height=250,
    aggrid_version="35.1.0",  # Pin to specific version
)
grid_versioned

In [ ]:
# Grid with fixed width (smaller than default)
grid_small = create_grid(
    df,
    column_defs=columns,
    height=250,
    width="1200px",  # Fixed width instead of 100%
)
grid_small

In [ ]:
# Grid with columns sized to fit the grid width
grid_fit = create_grid(
    df,
    column_defs=columns,
    height=250,
    size_columns_to_fit=True,  # Columns expand to fill grid width
)
grid_fit

In [ ]:
# Grid with auto-sizing disabled (columns use fixed widths)
# Note: auto_size_columns=True is the default
grid_no_autosize = create_grid(
    df,
    column_defs=columns,
    height=250,
    auto_size_columns=False,  # Disable auto-sizing, use fixed column widths
)
grid_no_autosize

## Grid within App

In [ ]:
def update_grid(app):
    with app.messages:
        app.status("Updating grid...")

        precision = app["precision"]

        # Generate column definitions and apply formatting
        cols = get_column_defs(df)
        cols[0]["headerName"] = "Company"
        cols[0]["pinned"] = "left"
        apply_format(cols[1], FORMAT_DECIMAL, precision=precision)
        cols[1]["headerName"] = "Price"
        apply_format(cols[2], FORMAT_PERCENT, precision=precision)
        cols[2]["headerName"] = "Change %"
        apply_format(cols[3], FORMAT_MAG_SI, precision=1)
        cols[3]["headerName"] = "Volume"
        apply_format(cols[4], FORMAT_MAG_SI, precision=2)
        cols[4]["headerName"] = "Market Cap"

        app.tab(0).clear().dataframe(
            "grid",
            df,
            column_defs=cols,
            on_click=lambda x: print(f"Clicked: {x}"),
            height=300,
        )

        app.done("Grid updated")


app = nb.App(tabs=["Grid"], footer="Messages", header="Config")

app.config.select("precision", options=[0, 1, 2, 3, 4], default=2, label="Precision")
app.config.button("update", on_click=update_grid, label="Update Grid", status=True)

update_grid(app)

app.display()